# fMRI pre-processing and atlas extraction
## A step-by-step tutorial
________________


This project was done during the Brain Hack School 2020 organized by McGill University, University of Montreal, Concordia University and Polytechnique. 

The goal of this tutorial is to generate functional connectivity matrices from functional magnetic resonance imaging (fMRI) starting from nifti images in BIDS format. The proposed pipeline first uses fMRIPrep to execute the pre-processing. Then, the Nilearn package is used to extract connectivity matrices from the pre-processed images using different brain parcellations. 

This project will be executed with data from the PREVENT-AD, a dataset of participants at risk of Alzheimer's Disease (AD), available openly on the Canadian Open Neuroscience Platform. We will use fMRI data from 10 participants at two different timepoints, at their baseline visit and at their 12 months visit, to obtain two functional connectivity matrices per participants.
________________


## Breakdown of the tutorial
### 1. - Presentation of the data (Prevent-AD / CONP)
### 2. - Preparation of the data for fMRIPrep
### 3. - Verifying the outputs of the preprocessing
### 4. - Preparing the data for Nilearn extraction
### 5. - Scripts for HPC processing

### 1. - Presentation of the data (Prevent-AD / CONP)

#### 1.1 - Preparation for data download

The data used for this project is taken from the Prevent-AD cohort, the data for which is part of the Canadian Open Neuroscience Platform (CONP). The data is accessible [here](https://portal.conp.ca/dataset?id=projects/preventad-open). Note that an account is required for accessing the data. You can gain access by filling the form [here](https://openpreventad.loris.ca/). If you are using your own dataset with fMRIPrep, you can skip ahead to section 2.0 of the tutorial.

You can refer to the instructions on the CONP/Prevent-AD for how to download the data. Note that you will require the following to be able to install the dataset on your workstation:
- Datalad (you can install Datalad from this [link](http://handbook.datalad.org/en/latest/intro/installation.html)
- Git-annex (you can install Git-Annex from this [link](https://git-annex.branchable.com/install/)
- Homebrew (in the case where you can't install Git-annex with Conda and you use a Mac, as was my case, you can install it using Homebrew. You can install Homebrew from this [link](https://brew.sh/)

With Datalad, you gain access to the full dataset through symbolic links. This means that you will have access to the folders and be able to see what you can download before downloading any actual data. You will need to enter your credentials for the Open PREVENT-AD initiative to download and work with anything. 

**Be careful:** Note that the full dataset is quite heavy (170.89GB). Datalad gives an option to download all subjects, but only subjects/sessions of interest should be downloaded.

The commands used in bash to download the data are provided below for illustration purposes:

In [8]:
#Move to the directory where you want to download the data (I created a special folder called data_CONP/)

%cd /Users/stong3/Desktop/data_CONP/
#!datalad install https://github.com/CONP-PCNO/conp-dataset.git

#Once installed, you can go in the directory and install the Prevent-AD dataset
%cd /Users/stong3/Desktop/data_CONP/conp-dataset/
#!datalad install projects/preventad-open

#You can now navigate to the project directory
%cd /Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open/

#You should be able to see the list of subjects within the project
!ls

/Users/stong3/Desktop/data_CONP
/Users/stong3/Desktop/data_CONP/conp-dataset
/Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open
1004359   2424540   3408795   4396879   5456920   6788676   7917918   9522570
1016072   2448082   3452929   4437799   5458966   6794127   7945015   9539210
1031654   2484374   3455156   4498598   5558904   6795892   8019820   9555827
1072774   2496306   3463254   4532706   5562282   6851811   8036701   9566680
1076159   2623146   3475739   4538817   5692079   6852929   8120729   9584420
1154932   2631883   3525262   4541749   5707288   6865719   8351740   9605091
1176949   2663318   3531206   4604101   5730499   6909468   8389790   9650197
1177880   2738676   3594014   4628515   5774407   7055352   8445201   9690995
1263509   2757160   3619481   4652225   5817042   7098391   8469881   9742226
1284264   2823276   3692881   4676902   5836679   7125565   8477651   9742435
1322140   2843957   3697357   4696223   5984591   7161471   8478383   9748

#### 1.2 - Data download

We can now see that we can access all the subjects within the Prevent-AD project. All the subjects are divided in the following manner: 
(INSERT IMAGE FROM SITE HERE)

For this pre-processing tutorial, we will only use 10 subjects from whom we have the Baseline and the 12 months follow-up. In total, we will have 20 fMRI scans to pre-process. For simplicity, I took the first 10 subjects where:
- An anatomical scan (T1w) was available at baseline
- A resting-state functional MRI (rest BOLD) was available at baseline
- An anatomical scan (T1w) was available at 12 months
- A resting-state functional MRI (rest BOLD) was available at 12 months

The way Datalad downloads data is by using the following command: ```datalad get <filepath>```. 

In our case, based on the folders, we will require the following subjects (datalad commands below) and will only download these to reduce the data to analyze:
- 1004359 (Done) 
```/Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open/1004359/```
- 1016072 (Done) 
```/Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open/1016072/```
- 1072774 (Done) 
```/Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open/1072774/```
- 1076159 (Done) 
```/Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open/1076159/```
- 1154932 (Done)
```/Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open/1154932/```
- 1176949 (Done)
```/Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open/1176949/```
- 1177880 (Done)
```/Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open/1177880/```
- 1284264 (Done)
```/Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open/1284264/```
- 1322140 (Done)
```/Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open/1322140/```
- 1346022 (Done)
```/Users/stong3/Desktop/data_CONP/conp-dataset/projects/preventad-open/1346022/```

The first time this is done, there will be a prompt to enter a username and password (twice). These are the same as the ones create to access LORIS.